In [1]:
import os
import random
import numpy as np
from PIL import Image
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset, Subset
import torch

# ----------------------------
# Setup data roots and randomness
# ----------------------------
train_dir = "C:/Users/ssowl/CodeIt/train_image/train"
test_dir  = "C:/Users/ssowl/CodeIt/test_image/test"

device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 32
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# ----------------------------
# 1) Apply downsampling to balance data
# ----------------------------
def balance_dataset(folder):
    class_folders = [f for f in os.listdir(folder) if os.path.isdir(os.path.join(folder, f))]
    
    # Take only image files
    class_files = {}
    for cls in class_folders:
        path = os.path.join(folder, cls)
        files = [os.path.join(path, f) for f in os.listdir(path) if f.lower().endswith((".jpg",".png",".jpeg",".bmp",".webp"))]
        class_files[cls] = files
    
    # Downsampling
    min_count = min(len(files) for files in class_files.values())
    
    balanced_files = []
    balanced_labels = []
    for idx, cls in enumerate(class_folders):
        sampled = random.sample(class_files[cls], min_count)
        balanced_files.extend(sampled)
        balanced_labels.extend([idx]*len(sampled))
    
    return balanced_files, balanced_labels, class_folders

# ----------------------------
# 2) Balance train / test data
# ----------------------------
train_files, train_labels, class_names = balance_dataset(train_dir)
test_files, test_labels, _ = balance_dataset(test_dir)

print("Number of sample for each class of train dataset:", {class_names[i]: train_labels.count(i) for i in range(len(class_names))})
print("Number of sample for each class of test dataset:", {class_names[i]: test_labels.count(i) for i in range(len(class_names))})

# ----------------------------
# 3) Define class
# ----------------------------
class CustomDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img = Image.open(self.file_paths[idx]).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

# ----------------------------
# 4) Transform
# ----------------------------
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# ----------------------------
# 5) Dataset & Dataloader
# ----------------------------
train_dataset = CustomDataset(train_files, train_labels, transform=train_transform)
test_dataset  = CustomDataset(test_files, test_labels, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("Balanced Train size:", len(train_dataset))
print("Balanced Test size:", len(test_dataset))


Number of sample for each class of train dataset: {'fake': 153, 'real': 153}
Number of sample for each class of test dataset: {'fake': 110, 'real': 110}
Balanced Train size: 306
Balanced Test size: 220


In [4]:
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
import random

# ----------------------------
# 1) Fix the randomness
# ----------------------------
# With the same data, the same model, and the same batch order, the training results are always identical
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# ----------------------------
# 2) Setup CPU or GPU
# ----------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"

# ----------------------------
# 3) Use the balanaced train/test datasets (photos) 
# ----------------------------

# ----------------------------
# 4) Split the validation dataset
# ----------------------------
val_ratio = 0.2
num_train = len(train_dataset)
num_val   = int(val_ratio * num_train)
num_train = num_train - num_val

train_dataset, val_dataset = torch.utils.data.random_split(
    train_dataset, [num_train, num_val]
)

val_dataset.dataset.transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3,[0.5]*3)
])

# Shuffle the batch order randomly during training to prevent overfitting
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ----------------------------
# 5) Define the model (ResNet50 + Dropout)
# ----------------------------
model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(0.5), # Prevent overfitting,
    nn.Linear(num_features, len(class_names))
)

# Fine-tuning range: layer3, layer4, fc
# Stable training on a small dataset
for name, param in model.named_parameters():
    if "layer3" in name or "layer4" in name or "fc" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

model = model.to(device)

# ----------------------------
# 6) Setup loss function, optimizer, and LR scheduler
# ----------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=2, verbose=True)

# ----------------------------
# 7) Setup learning loop and early stopping
# ----------------------------
epochs = 20
best_val_acc = 0.0
best_model_wts = copy.deepcopy(model.state_dict()) # Save the model with the highest validation accuracy
patience = 5
counter = 0
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step() # Update parameters
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)

    # Compute validation accuracy
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_acc = 100 * correct / total

    # Compute LR scheduler
    scheduler.step(val_acc)

    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}, Val Accuracy: {val_acc:.2f}%")

    # Setup early stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_wts = copy.deepcopy(model.state_dict())
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered!")
            break

# ----------------------------
# 8) Apply the best model
# ----------------------------
model.load_state_dict(best_model_wts)

# ----------------------------
# 9) Define an evaluation function
# ----------------------------
def evaluate(loader, dataset_name="Dataset"):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    print(f"{dataset_name} Accuracy: {acc:.2f}%")
    return acc

# ----------------------------
# 10) Print train / validation / test accuracy
# ----------------------------
evaluate(train_loader, "Train")
evaluate(val_loader, "Validation")
evaluate(test_loader, "Test")


C:\Users\ssowl\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ssowl\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\ssowl\anaconda3\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
C:\Users\ssowl\anaconda3\Lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.war

Epoch 1/20, Loss: 0.6868, Val Accuracy: 77.05%
Epoch 2/20, Loss: 0.2956, Val Accuracy: 81.97%
Epoch 3/20, Loss: 0.1012, Val Accuracy: 83.61%
Epoch 4/20, Loss: 0.0354, Val Accuracy: 80.33%
Epoch 5/20, Loss: 0.0124, Val Accuracy: 81.97%
Epoch 6/20, Loss: 0.0084, Val Accuracy: 80.33%
Epoch 7/20, Loss: 0.0054, Val Accuracy: 78.69%
Epoch 8/20, Loss: 0.0127, Val Accuracy: 80.33%
Early stopping triggered!
Train Accuracy: 100.00%
Validation Accuracy: 83.61%
Test Accuracy: 79.09%


79.0909090909091